In [1]:
########### change start up py
%run ../Startup.py

PID number is 8348


The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

# Load chromatin dataframe


Data below can be generated from the notebook as:

[preprocess/2_dna_merfish/scripts/2_spot_pick/4_summarize_jie_to_dict](../../preprocess/2_dna_merfish/scripts/2_spot_pick/4_summarize_jie_to_dict.ipynb)

In [2]:
df_cells = pd.read_csv(r"E:\DNA_analysis\Postanalysis_WT_MoP\selected_all_cells.csv")
df_cells.head()

,rna_experiment,uid,fov_id,cell_id,subclass,chr,hyb,fiberidx,numfiber,z_um,x_um,y_um
0,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,0,1,1,4.147276,3253.588485,-6244.868778
1,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,4,1,1,4.753624,3253.467060,-6243.619909
2,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,5,1,1,5.500038,3254.268482,-6244.335963
3,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,9,1,1,5.381981,3253.031182,-6243.768668
4,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,10,1,1,4.415525,3252.995276,-6243.899623


In [3]:
class_correspondance = {'Gluta':['L2/3 IT', 'L4/5 IT', 'L5 IT', 'L6 IT', 'L5 ET', 'L5/6 NP', 'L6 CT', 'L6b'],
                       'GABA':['Pvalb', 'Sst', 'Sncg', 'Lamp5', 'Vip'], 'Astro':['Astro'], 'Oligo':['Oligo', 'OPC'],
                       'Micro':['Micro'], 'Endo':['Endo', 'Peri', 'SMC', 'VLMC']}
subcls_to_cls = {}
for cls, subcls_list in class_correspondance.items():
    for subcls in subcls_list:
        subcls_to_cls[subcls] = cls

In [4]:
df_cells['majorType'] = df_cells['subclass'].apply(lambda x: subcls_to_cls[x])
df_cells.head()

,rna_experiment,uid,fov_id,cell_id,subclass,chr,hyb,fiberidx,numfiber,z_um,x_um,y_um,majorType
0,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,0,1,1,4.147276,3253.588485,-6244.868778,Oligo
1,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,4,1,1,4.753624,3253.467060,-6243.619909,Oligo
2,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,5,1,1,5.500038,3254.268482,-6244.335963,Oligo
3,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,9,1,1,5.381981,3253.031182,-6243.768668,Oligo
4,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,10,1,1,4.415525,3252.995276,-6243.899623,Oligo


# Calculate chromatin insulation

In [6]:
# prepare arguments
pos_cols = ['z_um', 'x_um', 'y_um']
info_cols = ['uid', 'majorType', 'subclass', 'chr', 'fiberidx']
sel_cols = info_cols + pos_cols

df_list = []
info_list = []

for _, df in tqdm(df_cells.groupby('uid')):
    df_list.append(df[sel_cols].copy())
    info_list.append(info_cols)

df_list[0]

100%|██████████████████████████████████████████████████████████████████████████| 46340/46340 [00:18<00:00, 2505.56it/s]


,uid,majorType,subclass,chr,fiberidx,z_um,x_um,y_um
29787278,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,6.923609,399.191452,4067.596867
29787279,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,7.105266,399.373386,4067.844072
29787280,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,7.428478,398.929961,4068.254104
29787281,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,8.313752,399.304615,4068.385576
29787282,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,8.156805,399.674011,4068.130243
...,...,...,...,...,...,...,...,...
29787994,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr9,1,7.992010,401.706214,4065.802888
29787995,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr9,1,7.494973,402.022222,4065.521632
29787996,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr9,1,6.911271,400.825881,4066.519783
29787997,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr9,1,8.104205,400.739625,4066.510990


Load function **chromatin_property_CW** from [chromatin_property_CW](../../functions/chromatin_property_CW.py)

In [7]:
import multiprocessing as mp
from ../../functions import chromatin_property_CW

In [13]:
with mp.Pool(16) as pool:
    results = pool.starmap(chromatin_property_CW.calculate_chr_insulation, zip(df_list[:],info_list[:]), chunksize=1)

df_insulation = pd.concat(results)
df_insulation.reset_index(inplace=True, drop=True)
df_insulation.head()

,uid,majorType,subclass,chr_1,chr_2,fiberidx_1,fiberidx_2,insulation_score
0,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,chr10,1.0,1.0,3.674562
1,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,chr11,1.0,1.0,2.155158
2,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,chr12,1.0,1.0,2.422183
3,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,chr13,1.0,1.0,2.060740
4,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,chr15,1.0,1.0,1.639320


In [14]:
############ define output 
data_folder = r'chromatin_property'
df_insulation_filename = os.path.join(data_folder, 'WT_MOp_insulation_all.csv')

if not os.path.exists(df_insulation_filename):
    df_insulation.to_csv(df_insulation_filename)

In [21]:
# get insulation score averaged by chromosomes
chroms = []
for i in range(1,20):
    chroms.append('chr'+str(i))
chroms.append('chrX')

# initialize
uids = []
majortypes = []
subclses = []
chrom_dfs = []
insulations = []

# construct dataframe
for uid, df_uid in tqdm(df_insulation.groupby('uid')):
    for chrom in chroms:
        df = df_uid[(df_uid['chr_1']==chrom)|(df_uid['chr_2']==chrom)].copy()
        if len(df)>0:
            uids.append(uid)
            majortypes.append(df['majorType'].values[0])
            subclses.append(df['subclass'].values[0])
            chrom_dfs.append(chrom)
            insulations.append(np.mean(df['insulation_score']))

df_insulation_summary = pd.DataFrame({'uid':uids, 'majorType':majortypes, 'subclass':subclses,
                                     'chr':chrom_dfs, 'mean_insulation_score':insulations})
df_insulation_summary.head()

100%|███████████████████████████████████████████████████████████████████████████| 42834/42834 [04:02<00:00, 176.51it/s]


,uid,majorType,subclass,chr,mean_insulation_score
0,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,2.595337
1,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr2,2.609158
2,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr3,2.274567
3,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr5,2.571502
4,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr7,2.239924


In [22]:
data_folder = r'E:\DNA_analysis\Postanalysis_WT_MoP\chromatin_property'
df_insulation_summary_filename = os.path.join(data_folder, 'WT_MOp_insulation_average.csv')

if not os.path.exists(df_insulation_summary_filename):
    df_insulation_summary.to_csv(df_insulation_summary_filename)